# Capstone Two: Data Wrangling

## 1. Data Collection


- `synthetic_invoices.csv`
- `synthetic_vials.csv`
- `synthetic_dispense_log.csv`
- `synthetic_claims.csv`
These files will be loaded into pandas DataFrames and then merged together.

In [4]:
import pandas as pd
import numpy as np

# Load datasets
try:
    invoices_df = pd.read_csv("synthetic_invoices.csv")
    vials_df = pd.read_csv("synthetic_vials.csv")
    dispense_log_df = pd.read_csv("synthetic_dispense_log.csv")
    claims_df = pd.read_csv("synthetic_claims.csv")
except FileNotFoundError as e:
    print(f"Error loading data: {e}. Please ensure all CSV files are in the correct directory.")
    raise

### Data Joining
The DataFrames are merged sequentially using left joins to create a comprehensive dataset. The process starts with `vials_df` and progressively incorporates invoice, dispense, and claim information.

In [6]:
# Merge datasets into a single DataFrame
vial_invoice_df = pd.merge(vials_df, invoices_df, on="Invoice Number", how="left")
full_df = pd.merge(vial_invoice_df, dispense_log_df, on="Vial Number", how="left")
full_df = pd.merge(full_df, claims_df, on="Dispense ID", how="left")

## 2. Data Organization

A well-organized file structure is crucial for project reproducibility and clarity. For this project, a dedicated directory has been created which contains:
- A `data` subdirectory for the raw CSV files.
- This Jupyter Notebook (`.ipynb`) for the data wrangling process.
- A `README.md` file explaining the project.

This entire project is managed using a **GitHub repository**. This is essential for version control, collaboration, and showcasing the work.

## 3. Data Definition

In this step, we'll gain a thorough understanding of the merged dataset's features. This involves examining column names, data types, value ranges, and summary statistics to identify any potential issues before cleaning.

In [11]:
full_df.head()

,Vial Number,Lot Number,Expiration Date,Purchase Price,Invoice Number,Invoice Date,Total Amount,Provider,Location,Dispense ID,Patient ID,Date Of Dispense,Username,Unscanned,Claim ID,Claim Date,Amount,Denied
0,VN-0000001,LOT-5169460,2026-04-30,$50.00,INV-000402,2025-07-28,902.06,Eye Care BigTown,BigTown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,VN-0000002,LOT-5228790,2026-04-30,$50.00,INV-000637,2025-06-15,2237.03,EyeCare Smallville,Smallville,DISP-000293,676660.0,2025-04-15,asha.lou,False,CLM-000293,2025-07-28,153.83,False
2,VN-0000003,LOT-5134343,2026-04-30,$50.00,INV-000757,2025-07-06,2480.64,EyeCare Smallville,BigTown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,VN-0000004,LOT-4417439,2026-04-30,$50.00,INV-000284,2025-04-26,2514.06,EyeCare Smallville,BigTown,DISP-004397,962805.0,2025-04-10,jen.kirby,False,CLM-004397,2025-06-19,116.12,False
4,VN-0000005,LOT-8583121,2026-04-30,$50.00,INV-000303,2025-02-12,4599.87,EyeCare Smallville,Smallville,DISP-006754,464413.0,2025-04-12,kasey.convertem,False,CLM-006754,2025-06-11,124.73,False


In [12]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15754 entries, 0 to 15753
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Vial Number       15754 non-null  object 
 1   Lot Number        15754 non-null  object 
 2   Expiration Date   15754 non-null  object 
 3   Purchase Price    15754 non-null  object 
 4   Invoice Number    15754 non-null  object 
 5   Invoice Date      15754 non-null  object 
 6   Total Amount      15754 non-null  float64
 7   Provider          15754 non-null  object 
 8   Location          15754 non-null  object 
 9   Dispense ID       11000 non-null  object 
 10  Patient ID        11000 non-null  float64
 11  Date Of Dispense  11000 non-null  object 
 12  Username          11000 non-null  object 
 13  Unscanned         11000 non-null  object 
 14  Claim ID          11000 non-null  object 
 15  Claim Date        11000 non-null  object 
 16  Amount            11000 non-null  float6

**Initial Observations from `.info()`:**
1.  **Missing Values**: There are significant missing values (15746 - 10992 = 4754) in columns related to dispense and claim events. This is expected, as not all purchased vials have been dispensed.
2.  **Incorrect Data Types**:
    - `Purchase Price` is an `object` type due to the '$' symbol and should be a numeric type (`float`).
    - `Date of Purchase`, `Date Of Dispense`, and `Claim Date` are `object` types and should be converted to `datetime` for time-series analysis.
    - `Patient ID` is a `float`, which is unusual for an identifier. It should likely be an `integer` or `string`.
    - `Unscanned` and `Denied` are `object` types but represent boolean states. They should be converted to the `bool` type.

In [14]:
# Summary statistics for numerical columns
full_df.describe()

,Total Amount,Patient ID,Amount
count,15754.000000,11000.000000,11000.000000
mean,2746.863912,550948.549818,130.446065
std,1279.527817,259157.789564,40.482171
min,500.170000,100033.000000,60.010000
25%,1646.940000,325000.250000,95.630000
50%,2758.380000,552737.000000,130.290000
75%,3820.770000,775434.750000,165.905000
max,4998.820000,999868.000000,199.980000


In [15]:
# Summary statistics for object columns
full_df.describe(include='object')

,Vial Number,Lot Number,Expiration Date,Purchase Price,Invoice Number,Invoice Date,Provider,Location,Dispense ID,Date Of Dispense,Username,Unscanned,Claim ID,Claim Date,Denied
count,15754,15754,15754,15754,15754,15754,15754,15754,11000,11000,11000,11000,11000,11000,11000
unique,12000,11998,1,1,1200,242,2,2,11000,216,5,2,11000,216,2
top,VN-0004325,LOT-4069604,2026-04-30,$50.00,INV-000756,2025-04-30,Eye Care BigTown,BigTown,DISP-000293,2025-03-18,kasey.convertem,False,CLM-000293,2025-06-17,False
freq,6,6,15754,15754,33,151,7878,8270,1,71,2248,10378,1,69,10594


## 4. Data Cleaning

Based on the data definition step, the following actions are required:
1.  Correct the data types for `Purchase Price`, all date columns, `Patient ID`, `Unscanned`, and `Denied`.
2.  Check for and handle any duplicate rows.
3.  Address the missing values in a logical manner.

In [18]:
# 1. Correct Data Types
# Clean and convert 'Purchase Price' to float
full_df['Purchase Price'] = full_df['Purchase Price'].replace({'\$': ''}, regex=True).astype(float)

<>:3: SyntaxWarning: invalid escape sequence '\$'
<>:3: SyntaxWarning: invalid escape sequence '\$'
C:\Users\rahul\AppData\Local\Temp\ipykernel_5740\1813675566.py:3: SyntaxWarning: invalid escape sequence '\$'
  full_df['Purchase Price'] = full_df['Purchase Price'].replace({'\$': ''}, regex=True).astype(float)


In [23]:
# Convert date columns to datetime objects
date_columns = ['Expiration Date', 'Date Of Dispense', 'Claim Date','Invoice Date']
for col in date_columns:
    full_df[col] = pd.to_datetime(full_df[col], errors='coerce')

In [25]:
# Convert boolean-like columns to boolean type
bool_columns = ['Unscanned', 'Denied']
for col in bool_columns:
    full_df[col] = full_df[col].astype('boolean')

In [27]:
# Convert 'Patient ID' to a nullable integer type
full_df['Patient ID'] = full_df['Patient ID'].astype('Int64')

In [29]:
print("Data types corrected successfully.")
full_df.info()

Data types corrected successfully.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15754 entries, 0 to 15753
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Vial Number       15754 non-null  object        
 1   Lot Number        15754 non-null  object        
 2   Expiration Date   15754 non-null  datetime64[ns]
 3   Purchase Price    15754 non-null  float64       
 4   Invoice Number    15754 non-null  object        
 5   Invoice Date      15754 non-null  datetime64[ns]
 6   Total Amount      15754 non-null  float64       
 7   Provider          15754 non-null  object        
 8   Location          15754 non-null  object        
 9   Dispense ID       11000 non-null  object        
 10  Patient ID        11000 non-null  Int64         
 11  Date Of Dispense  11000 non-null  datetime64[ns]
 12  Username          11000 non-null  object        
 13  Unscanned         11000 non-null  boolean

In [31]:
# 2. Check for duplicates
duplicates = full_df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

if duplicates > 0:
    full_df = full_df.drop_duplicates()
    print("Duplicates removed.")
else:
    print("No duplicates found.")

Number of duplicate rows: 0
No duplicates found.


In [33]:
# 3. Handle missing values
missing_values = full_df.isnull().sum()
print("Missing values before handling:")
print(missing_values[missing_values > 0])

# For vials that were never dispensed, it's logical that dispense/claim info is missing.
# We will fill boolean columns with False and numerical 'Amount' with 0.
# Other identifiers (like Dispense ID, Patient ID) will be left as NaN (or NaT for dates) as they represent a non-event.
full_df['Unscanned'].fillna(False, inplace=True)
full_df['Denied'].fillna(False, inplace=True)
full_df['Amount'].fillna(0, inplace=True)

print("Missing values handled appropriately.")

Missing values before handling:
Dispense ID         4754
Patient ID          4754
Date Of Dispense    4754
Username            4754
Unscanned           4754
Claim ID            4754
Claim Date          4754
Amount              4754
Denied              4754
dtype: int64
Missing values handled appropriately.


C:\Users\rahul\AppData\Local\Temp\ipykernel_5740\3758179693.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  full_df['Unscanned'].fillna(False, inplace=True)
C:\Users\rahul\AppData\Local\Temp\ipykernel_5740\3758179693.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exam

## 5. Analysis and Reporting

With the data now wrangled and cleaned, we can perform the final analysis to derive insights.

In [37]:
# Analyze and report
print("--- Ophthalmology Drug Tracking Analysis ---")

# 1. Identify Unscanned Vials
unscanned_vials = full_df[full_df['Unscanned'] == True]
print(f"\n## Unscanned Vials Report ({len(unscanned_vials)} found)")
if not unscanned_vials.empty:
    print("The following vials were dispensed but not scanned, leading to potential revenue loss:")
    print(unscanned_vials[['Vial Number', 'Lot Number', 'Purchase Price', 'Dispense ID', 'Patient ID', 'Date Of Dispense', 'Username']])
else:
    print("No unscanned vials found.")

--- Ophthalmology Drug Tracking Analysis ---

## Unscanned Vials Report (622 found)
The following vials were dispensed but not scanned, leading to potential revenue loss:
      Vial Number   Lot Number  Purchase Price  Dispense ID  Patient ID  \
6      VN-0000005  LOT-8583121            50.0  DISP-010328      567386   
34     VN-0000025  LOT-5251569            50.0  DISP-003474      930415   
50     VN-0000038  LOT-1985938            50.0  DISP-002492      435402   
71     VN-0000050  LOT-8239582            50.0  DISP-007851      722927   
77     VN-0000056  LOT-1894126            50.0  DISP-000043      702136   
...           ...          ...             ...          ...         ...   
15618  VN-0011891  LOT-1987565            50.0  DISP-000017      516499   
15639  VN-0011906  LOT-6786924            50.0  DISP-000179      394925   
15643  VN-0011910  LOT-7320978            50.0  DISP-005509      882115   
15703  VN-0011959  LOT-3824453            50.0  DISP-006094      820014   
1573

In [39]:
# 2. Identify Denied Claims
denied_claims = full_df[full_df['Denied'] == True]
print(f"\n## Denied Claims Report ({len(denied_claims)} found)")
if not denied_claims.empty:
    print("The following claims were denied by insurance:")
    print(denied_claims[['Claim ID', 'Dispense ID', 'Vial Number', 'Patient ID', 'Claim Date', 'Amount']])
else:
    print("No denied claims found.")


## Denied Claims Report (406 found)
The following claims were denied by insurance:
         Claim ID  Dispense ID Vial Number  Patient ID Claim Date  Amount
6      CLM-010328  DISP-010328  VN-0000005      567386 2025-09-09   92.40
50     CLM-002492  DISP-002492  VN-0000038      435402 2025-03-26  102.98
71     CLM-007851  DISP-007851  VN-0000050      722927 2025-02-22  119.08
77     CLM-000043  DISP-000043  VN-0000056      702136 2025-05-07   65.80
201    CLM-008965  DISP-008965  VN-0000149      406197 2025-07-21  124.17
...           ...          ...         ...         ...        ...     ...
15574  CLM-006701  DISP-006701  VN-0011860      592974 2025-04-04  115.14
15580  CLM-000525  DISP-000525  VN-0011864      720346 2025-08-23  146.53
15583  CLM-005342  DISP-005342  VN-0011867      409574 2025-04-21  155.99
15643  CLM-005509  DISP-005509  VN-0011910      882115 2025-03-19  174.90
15703  CLM-006094  DISP-006094  VN-0011959      820014 2025-07-11  180.63

[406 rows x 6 columns]


In [41]:
# 3. Profitability Analysis
# A reimbursed claim is one that has a Claim ID, was not denied, and has a positive amount.
reimbursed_claims = full_df[(full_df['Claim ID'].notna()) & (full_df['Denied'] == False)].copy()
reimbursed_claims['Profit'] = reimbursed_claims['Amount'] - reimbursed_claims['Purchase Price']

total_revenue = reimbursed_claims['Amount'].sum()
total_cost = reimbursed_claims['Purchase Price'].sum()
total_profit = reimbursed_claims['Profit'].sum()

print("\n## Profitability Analysis")
print(f"Total Revenue from Reimbursed Claims: ${total_revenue:,.2f}")
print(f"Total Cost of Goods Sold: ${total_cost:,.2f}")
print(f"Total Profit: ${total_profit:,.2f}")

if not reimbursed_claims.empty:
    print("\n### Top 5 Most Profitable Vials:")
    print(reimbursed_claims.nlargest(5, 'Profit')[['Vial Number', 'Purchase Price', 'Amount', 'Profit']])
    print("\n### Top 5 Least Profitable Vials:")
    print(reimbursed_claims.nsmallest(5, 'Profit')[['Vial Number', 'Purchase Price', 'Amount', 'Profit']])


## Profitability Analysis
Total Revenue from Reimbursed Claims: $1,381,695.61
Total Cost of Goods Sold: $529,700.00
Total Profit: $851,995.61

### Top 5 Most Profitable Vials:
      Vial Number  Purchase Price  Amount  Profit
15     VN-0000010            50.0  199.98  149.98
2695   VN-0002064            50.0  199.96  149.96
13870  VN-0010556            50.0  199.94  149.94
635    VN-0000473            50.0  199.91  149.91
13145  VN-0010004            50.0  199.91  149.91

### Top 5 Least Profitable Vials:
      Vial Number  Purchase Price  Amount  Profit
14142  VN-0010762            50.0   60.01   10.01
6071   VN-0004649            50.0   60.03   10.03
7035   VN-0005378            50.0   60.03   10.03
14811  VN-0011287            50.0   60.03   10.03
10549  VN-0008048            50.0   60.09   10.09
